In [ ]:
from lec_utils import *


<div class="alert alert-info" markdown="1">

#### Lecture 14

# Supplemental Notebook: Regression using Linear Algebra

### EECS 398: Practical Data Science, Spring 2025

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/sp25">github.com/practicaldsc/sp25</a> • 📣 See latest announcements [**here on Ed**](https://edstem.org/us/courses/78535/discussion/6647877) </small>
    
</div>


<div class="alert alert-warning" markdown="1">

**Note**: Most of Lecture 14 can be found in the PDF slides posted on the course website. This is a supplementary notebook with a few code demos.

</div>

## Understanding the Data

---

Let's load in the commute times dataset as a DataFrame.

In [ ]:
df = pd.read_csv('data/commute-times.csv')
df.head()

There are many columns in here, but the only ones we're interested in for now are `'departure_hour'` and `'minutes'`.

In [ ]:
df[['departure_hour', 'minutes']]

In [ ]:
fig = px.scatter(df,
           x='departure_hour',
           y='minutes',
           size=np.ones(len(df)) * 50,
           size_max=8)
fig.update_xaxes(title='Home Departure Time (AM)')
fig.update_yaxes(title='Minutes')
fig.update_layout(title='Commuting Time vs. Home Departure Time')
fig.update_layout(width=700)

## Implementing $w_0^*$ and $w_1^*$

---

Let's implement the formulas for the best slope, $w_1^*$, and intercept, $w_0^*$, we found in the lecture slides:

\begin{align*}
w_1^* &=
    \frac{
        \displaystyle
        \sum_{i=1}^n (x_i - \bar x)(y_i - \bar y)
    }{
        \displaystyle
        \sum_{i=1}^n (x_i - \bar x)^2
    }
    \qquad
    \qquad
w_0^* =
    \bar y - w_1^* \bar x
\end{align*}

In [ ]:
def slope(x, y):
    # Assume x and y are two Series.
    numerator = ((x - np.mean(x)) * (y - np.mean(y))).sum()
    denominator = ((x - np.mean(x)) ** 2).sum()
    return numerator / denominator
def intercept(x, y):
    return y.mean() - slope(x, y) * x.mean()

In [ ]:
w1_star = slope(df['departure_hour'], df['minutes'])
w1_star

In [ ]:
w0_star = intercept(df['departure_hour'], df['minutes'])
w0_star

The results above tell us that the linear hypothesis function with the lowest mean squared error on our dataset is:

$$\text{predicted commute time (minutes)} = 142.45 - 8.19 \cdot \text{departure hour}$$

We can use it to make predictions:

In [ ]:
def predict_commute(x_new):
    return w0_star + w1_star * x_new

What if I leave at 8AM? 10:45AM?

In [ ]:
predict_commute(8)

In [ ]:
predict_commute(10 + 45 / 60)

What do all of our predictions look like?

In [ ]:
hline = px.line(x=[5.5, 11.5], y=[predict_commute(5.5), predict_commute(11.5)]).update_traces(line={'color': 'red', 'width': 4})
fline1 = go.Figure(fig.data + hline.data)
fline1.update_xaxes(title='Home Departure Time (AM)')
fline1.update_yaxes(title='Minutes')
fline1.update_layout(title='<span style="color:red">Predicted Commute Time</span> = 142.25 - 8.19 * Departure Hour')
fline1.update_layout(width=700, margin={'t': 60})

### Aside: What does $R_{\text{sq}}(w_0, w_1)$ look like?

Let's draw a plot of $R_{\text{sq}}(w_0, w_1)$, the empirical risk that we're trying to minimize.
- When we only had a single parameter, $h$, $R(h)$ was in 2D.
    - One axis for $h$, one axis for $R(h)$.
- Now that we have two parameters, $w_0$ and $w_1$, $R(w_0, w_1)$ will be in 3D!
    - One axis for $w_0$, one axis for $w_1$, one axis for $R(w_0, w_1)$.
    - The bottom plane consists of all possible combinations of slope and intercept.
    - The height of the function above any pair of points on the bottom plane represents the MSE for that combination of slope and intercept.

In [ ]:
def mse(y_actual, y_pred):
    return np.mean((y_actual - y_pred)**2)
def mse_for_departure_model(w):
    w0, w1 = w
    return mse(df['minutes'], w0 + w1 * df['departure_hour'])
num_points = 50 # increase for better resolution, but it will run more slowly. 
# if (num_points <= 100):
uvalues = np.linspace(90, 190, num_points)
vvalues = np.linspace(-13, -3, num_points)
(u,v) = np.meshgrid(uvalues, vvalues)
thetas = np.vstack((u.flatten(),v.flatten()))
MSE = np.array([mse_for_departure_model(t) for t in thetas.T])
loss_surface = go.Surface(x=u, y=v, z=np.reshape(MSE, u.shape))
minimizer = go.Scatter3d(x=[w0_star], y=[w1_star], z=[mse_for_departure_model([w0_star, w1_star])], 
                         mode='markers', name='optimal parameters',
                         marker=dict(size=10, color='gold'))
fig = go.Figure(data=[loss_surface, minimizer])
# fig.add_trace(opt_point)
fig.update_layout(title='Loss Surface', scene = dict(
    xaxis_title = "w0",
    yaxis_title = "w1",
    zaxis_title = r"R(w0, w1)"))
fig.show()
# else:
#     print("Picking num points > 100 can be really slow. If you really want to try, edit the code above so that this if statement doesn't trigger.")

In Lecture 12, we used partial derivatives to minimize the graph above!

## Correlation

---

$$\begin{align*} r &= \text{the average of the product of $x$ and $y$, when both are standardized} \\ &= \frac{1}{n} \sum_{i = 1}^n \left( \frac{x_i - \bar{x}}{\sigma_x} \right) \left( \frac{y_i - \bar{y}}{\sigma_y} \right)  \end{align*}$$

In [ ]:
def correlation(x, y): 
    ...

In [ ]:
correlation(df['departure_hour'], df['minutes'])

In [ ]:
# Symmetric!
correlation(df['minutes'], df['departure_hour'])

In [ ]:
# Doesn't change if we multiply x or y by constants!
correlation(df['departure_hour'] * 1000, df['minutes'] * 545)

In [ ]:
# DataFrames have a built-in correlation method.
df[['departure_hour', 'minutes']].corr()

In [ ]:
# numpy has a built-in corrcoef method.
np.corrcoef(df['departure_hour'], df['minutes'])

## Implementing $w_0^*$ and $w_1^*$, Again

---

Recall, the formulas for the optimal intercept and slope are:

$$w_1^* = r \frac{\sigma_y}{\sigma_x}$$

$$w_0^* = \bar{y} - w_1^* \bar{x}$$

Let's define two new functions, `slope_again` and `intercept_again`, which use these slightly updated formulas. (Really, only the formula for $w_1^*$ has changed.)

In [ ]:
def slope_again(x, y):
    return correlation(x, y) * np.std(y) / np.std(x)

In [ ]:
def intercept_again(x, y):
    return y.mean() - slope_again(x, y) * x.mean()

In [ ]:
w1_star_again = slope_again(df['departure_hour'], df['minutes'])
w1_star_again

In [ ]:
w0_star_again = intercept_again(df['departure_hour'], df['minutes'])
w0_star_again

We get the same optimal intercept and slope as before!

In [ ]:
# From before:
(w1_star, w0_star)

In [ ]:
# Now:
(w1_star_again, w0_star_again)

## Implementing $w_0^*$ and $w_1^*$ using `sklearn`

---

In practice, you wouldn't manually implement formulas for $w_0^*$ and $w_1^*$. Instead, you'd use a pre-built implementation.

The Python package we'll use for machine learning is `sklearn`. We'll start seeing it more starting next lecture.

In [ ]:
from sklearn.linear_model import LinearRegression

To build a linear regression model that we can use for prediction, we first need to instantiate a `LinearRegression` object.

In [ ]:
model = LinearRegression()

Then, we need to `fit` the model by telling it what our $x$'s and $y$'s are.

In [ ]:
model.fit(X=df[['departure_hour']], y=df['minutes'])

Once the model is fit, we can look at its `intercept_` and `coef_` attributes to see $w_0^*$ and $w_1^*$, respectively.

In [ ]:
model.intercept_

In [ ]:
model.coef_

These are **exactly the same** as we found with our manual calculations! This means that `sklearn` is doing the same three step modeling process that we outlined in lecture.

Now that `model` is fit, we can use it for making predictions:

In [ ]:
model.predict([[8]])

In [ ]:
# Using our hand-build predict_commute function from earlier in the lecture:
predict_commute(8)

## Finding the Regression Line, Using the Normal Equations

---

Using our linear algebraic formulation, the optimal intercept and slope are given by the vector $\vec{w}^*$, where:

$$\vec{w}^* = ({X^TX})^{-1} X^T\vec{y}$$

Here:
- $X$ is a $n \times 2$ matrix, called the **design matrix**, defined as:

$${ X} = \begin{bmatrix} { 1} & { x_1} \\ { 1} & { x_2} \\ \vdots & \vdots \\ { 1} & { x_n} \end{bmatrix}$$

- $\vec{y}$ is a $n$-dimensional vector, called the **observation vector**, defined as:

$$\vec{y} = \begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}$$

Let's construct $X$ and $y$ in code.

First, the design matrix.

In [ ]:
# Create a new DataFrame by taking the 'departure_hour' column from df.
X = df[['departure_hour']].copy()
X

In [ ]:
# Add a column of all 1s to X.
X['1'] = 1
X

In [ ]:
# Change the order of the columns and convert to an array.
X = X[['1', 'departure_hour']].to_numpy()
X

$\vec{y}$ is already created for us: it's just the `'minutes'` column in `df`.

In [ ]:
y = df['minutes'].to_numpy()
y

Now, let's implement:

$$\vec{w}^* = ({X^TX})^{-1} X^T\vec{y}$$

In [ ]:
# The @ symbol performs matrix multiplication!
w_star_linalg = ...
w_star_linalg

These numbers look familiar!

In [ ]:
# Old formulas.
w0_star, w1_star

Indeed, they're exactly the same as the `w0_star` and `w1_star` we found using our old formulas.

Here's another way to find $\vec w^*$ using `X` and `y`. Rather than telling `numpy` to invert $X^TX$, we can tell it, literally, to solve the normal equations $X^TX \vec w = X^T \vec y$:

In [ ]:
np.linalg.solve(X.T @ X, X.T @ y)

## Making Predictions

---

We know how to make predictions with the old formulas:

In [ ]:
# The predicted commute time if I leave at 8:30AM.
...

How do we make predictions with the new formulas?

To find the predicted commute time for every departure hour in our dataset, we can multiply $X$ by the optimal parameter vector, $\vec{w}^*$.

$$\vec{h}^* = X \vec{w}^*$$

$\vec{h}^*$ above is the optimal **hypothesis vector**.

In [ ]:
all_preds = ...
all_preds

To make a prediction for a single data point, we must take the **dot product** of the optimal parameter vector, $\vec{w}^*$ (`w_star_linalg`) with a vector of the form $\begin{bmatrix} 1 & x_\text{new} \end{bmatrix}^T$, since this is what the rows of $X$ look like.

In [ ]:
# Also the predicted commute time if I leave at 8:30AM.
...

This gives us the same prediction as before!